In [ ]:
using Pkg
Pkg.activate("NB03")
Pkg.resolve()

In [ ]:
using Graphs, DataFrames, SparseArrays, LinearAlgebra, Plots
using GraphPlot, Colors
using VegaLite, VegaDatasets

Dieses Notebook basiert zu Teilen auf dem Kurs [DataScience](https://github.com/JuliaAcademy/DataScience) der JuliaAcademy von [Huda Nassar](https://github.com/nassarhuda). 

---
---
# Laden der Daten

Wir laden den `flights-airport` Datensatz:

In [ ]:
flights = dataset("flights-airport") |> DataFrame;

Die ersten 8 Zeilen dieses `DataFrames` sind wie folgt:

In [ ]:
flights[1:8,:]

Wir extrahieren die IDs der Flughäfen in einem Array `airports`.

In [ ]:
list_airports = vcat(flights[:,:origin],flights[:,:destination])
airports = unique(list_airports)
airports[1:5]

---
---
# Definition des Graphen

Als nächstes berechnen wir die Adjanzenzmatrix `A`.

In [ ]:
# build the adjacency matrix
n = length(airports)
A = spzeros(n,n)
for k = 1:size(flights, 1)
    i = findfirst(airports .== flights[k,:origin])
    j = findfirst(airports .== flights[k,:destination])
    A[i,j] = 1
end
A = max.(A,A')

Der Graph `G` wird durch Angabe von `A` definiert.

In [ ]:
G = Graph(A)

---
---
# Visualisierung

Wir schauen uns eine Visualisierung des Graphen an.

In [ ]:
layout= x -> spring_layout(x; C=20)
gplot(G, layout = layout,
        nodefillc = colorant"steelblue",
        linetype="curve"
)

Es scheint als wäre die Verteilung der Grade der blauen Knoten nicht gleichmäßig. Dies wird durch die folgendens Plots bestätigt.

In [ ]:
degrees = degree(G)
p1 = plot(sort(degrees,rev=true), ylabel="log degree", legend=false, yaxis=:log, title="Grad Verteilung")
p2 = plot(sort(degrees,rev=true), ylabel="degree", legend=false)
plot(p1, p2, size=(600,200))

---
---
# Geographische Visualisierung

Die erste Visualisierung hat `G` als Graphen ohne weitere Struktur verstanden. 

Die Knoten in `G` können mit Koordinaten in $\mathbb R^2$ identifiziert werden. 

Diese zusätzlich Struktur können wir visuell darstellen.

Zunächst laden wir einen Datensatz, der Längen- und Breitengrade der Flughäfen bereit stellt.

In [ ]:
all_locations = dataset("airports") |> DataFrame
k = [id in airports for id in all_locations[:,:iata]]
locations = all_locations[k,:]
locations[1:5,:]

Mit Hilfe des `VegaLite` Packages können wir dann den Graphen visualisieren:

In [ ]:
us10m = dataset("us-10m")
@vlplot(width=800, height=500) +
@vlplot(
    mark={:geoshape, fill=:lightgray, stroke=:white},
    data={values=us10m, format={type=:topojson,feature=:states}},
    projection={type=:albersUsa}) +
@vlplot(
    :circle,
    data=locations, projection={type=:albersUsa}, 
    longitude="longitude:q", latitude="latitude:q", 
    size={value=40}, color={value=:steelblue})+
@vlplot(
    mark={:rule, strokeWidth = 0.01,  opacity=0.2, color = "grey"},
    data=flights, 
         transform=[
                {lookup=:origin, 
                 from={data=locations, key=:iata, fields=["latitude", "longitude"]},
                 as=["origin_latitude", "origin_longitude"]},
                {lookup=:destination,
                 from={data=locations, key=:iata, fields=["latitude", "longitude"]},
                 as=["dest_latitude", "dest_longitude"]}],
        projection={type=:albersUsa},
        longitude="origin_longitude:q", latitude="origin_latitude:q",
        longitude2="dest_longitude:q", latitude2="dest_latitude:q")

---
---
# Berechnung von PageRank

Wir berechnen den PageRank jedes Flughafens in unserem Netzwerk $G=(V,E)$.

Erinnerung: die PageRank Verteilung ist durch den Eigenvektor zum Eigenwert 1 der Matrix 
$$ P = (p_{u,v})\in \mathbb R^{\vert V\vert \times \vert V\vert} , \quad p_{u,v}=\begin{cases} 1/\mathrm{deg}(v), & \{u,v\}\in E\\ 0, &\text{sonst}\end{cases}.
$$
gegeben.

Zunächst definieren wir $P$.

In [ ]:
P = zeros(n, n);
for u in 1:n
    for v in 1:n
        if A[u,v] == 1
            P[u,v] = 1/degrees[v]
        end
    end
end

Dann berechnen wir seine Eigenvektoren.

In [ ]:
using LinearAlgebra
E = eigen(P)
E.values

Der gesuchte Eigenvektor ist die letzte Spalte von `E.vector`.

In [ ]:
v = real.(E.vectors[:,end])
# test:
norm(P*v-v)

Für PageRank erstellenw wir ein neues `DataFrame`:

In [ ]:
PageRank =  DataFrame(airport = locations[:,:name], 
                      city = locations[:,:city],  
                      rank = v ./ sum(v),
                      longitude = locations[:,:longitude],
                      latitude = locations[:,:latitude]
)
PageRank[1:5,:]

Wir schauen, welche Flughäfen die größte PageRanks haben.

In [ ]:
PageRank_sorted = sort(PageRank,[:rank],rev=true)
PageRank_sorted[1:5,:]

---
---
# Visualisierung


Wir können nun die Graphen wie zuvor visualisieren, aber die Knoten ihrem PageRank entsprechend skalieren.

In [ ]:
layout= x -> spring_layout(x; C=20)
gplot(G, layout = layout,
        nodefillc = colorant"steelblue",
        linetype = "curve",
        nodesize = PageRank[:,:rank]
)

Wir sehen, dass der PageRank sich auf einige Flughäfen konzentriert.

In [ ]:
p1 = plot(PageRank_sorted[:,:rank], ylabel="log degree", legend=false, yaxis=:log, title="PageRank Verteilung")
p2 = plot(PageRank_sorted[:,:rank], ylabel="degree", legend=false)
plot(p1, p2, size=(600,200))

Die zweite Visualisierung ist wie folgt:

In [ ]:
@vlplot(width=800, height=500) +
@vlplot(
    mark={:geoshape, fill=:lightgray, stroke=:white},
    data={values=us10m, format={type=:topojson,feature=:states}},
    projection={type=:albersUsa}) +
@vlplot(
    :circle,
    data=PageRank, projection={type=:albersUsa}, 
    longitude="longitude:q", latitude="latitude:q", 
    size="rank", color={value=:steelblue})+
@vlplot(
    mark={:rule, strokeWidth = 0.01,  opacity=0.2, color = "grey"},
    data=flights, 
         transform=[
                {lookup=:origin, 
                 from={data=locations, key=:iata, fields=["latitude", "longitude"]},
                 as=["origin_latitude", "origin_longitude"]},
                {lookup=:destination,
                 from={data=locations, key=:iata, fields=["latitude", "longitude"]},
                 as=["dest_latitude", "dest_longitude"]}],
        projection={type=:albersUsa},
        longitude="origin_longitude:q", latitude="origin_latitude:q",
        longitude2="dest_longitude:q", latitude2="dest_latitude:q")

---
---
# Markov Prozesse
Wir wollen PageRank mit Hilfe eines Markov Prozesses approximieren.

Dazu samplen wir zunächst einen zufälligen Flughafen `X`.

In [ ]:
X = rand(1:n)
locations[X,:name]

Von `X` aus wollen wir zufällig zu einem weiteren Flughafen gehen. 

Die Zufallsverteilung wird dabei durch die zu `X`gehörende Spalte von `P` bestimmt.

In [ ]:
sum(P[:,X])

Wir wählen den Flughafen `Y` mit Wahrscheinlichkeit `P[Y,X]` aus. 

Dazu definieren wir eine Funktion, die diese Zufallswahl ausführt.

In [ ]:
function step(X, P)
    F = cumsum(P[:,X])
    r = rand()
    findfirst(F .> r)
end

Ein Zufallsprozess mit 10 Schritten kann dann wie folgt ausgeführt werden:

In [ ]:
ℓ = 10
walk = zeros(Int, ℓ)
walk[1] = X
for i in 2:ℓ
    walk[i] = step(walk[i-1], P)
end
locations[walk, :name]

Jeder Zufallsprozess mit $\ell$ Schritten definiert einen Zufallsverteilung auf $G$. 

Wir definieren eine Funktion, die von dieser Verteilung zieht:

In [ ]:
function sample(ℓ, P)
    X = rand(1:n)
    for i in 2:ℓ
        X = step(X, P)
    end
   X
end

und samplen einen Flughafen von dieser Verteilung:

In [ ]:
X = sample(10, P)
locations[X, :name]

wir haben nun eine Methode Zufallsobjekte zu generieren, deren Verteilung nahe bei der PageRank Verteilung liegt. Wir können so PageRank approximieren ohne Eigenwerte zu berechnen!

In [ ]:
Ω = [sample(20, P) for _ in 1:1e4];
p = [count(Ω .== i) for i in 1:n]
PageRankApprox =  DataFrame(airport = locations[:, :name], 
                      city = locations[:, :city],  
                      rank =  p./sum(p),
                      longitude = locations[:, :longitude],
                      latitude = locations[:, :latitude]
)
PageRankApprox[1:5,:]

Hier ist eine Visualisierung unserer Approximation:

In [ ]:
@vlplot(width=800, height=500) +
@vlplot(
    mark={:geoshape, fill=:lightgray, stroke=:white},
    data={values=us10m, format={type=:topojson,feature=:states}},
    projection={type=:albersUsa}) +
@vlplot(
    :circle,
    data=PageRankApprox, projection={type=:albersUsa}, 
    longitude="longitude:q", latitude="latitude:q", 
    size="rank", color={value=:steelblue})+
@vlplot(
    mark={:rule, strokeWidth = 0.01,  opacity=0.2, color = "grey"},
    data=flights, 
         transform=[
                {lookup=:origin, 
                 from={data=locations, key=:iata, fields=["latitude", "longitude"]},
                 as=["origin_latitude", "origin_longitude"]},
                {lookup=:destination,
                 from={data=locations, key=:iata, fields=["latitude", "longitude"]},
                 as=["dest_latitude", "dest_longitude"]}],
        projection={type=:albersUsa},
        longitude="origin_longitude:q", latitude="origin_latitude:q",
        longitude2="dest_longitude:q", latitude2="dest_latitude:q")